In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set('spark.app.name','pyspark_parse')
conf.set('spark.master','local[*]')

spark = SparkSession.builder\
    .config(conf=conf)\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/05 01:52:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/05 01:52:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/05 01:52:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
!ls -tl

total 1096
-rw-r--r--@ 1 giho.han  staff    4115 Jul  5 01:53 String_parse.ipynb
-rw-r--r--@ 1 giho.han  staff    8893 Jul  5 01:43 colab_main2.ipynb
drwxr-xr-x@ 2 giho.han  staff      64 Jul  5 01:43 spark-warehouse
-rw-r--r--@ 1 giho.han  staff   26077 Jul  5 01:18 colab_main1.ipynb
-rw-r--r--@ 1 giho.han  staff      10 Jul  5 00:10 README.md
-rw-r--r--@ 1 giho.han  staff     997 Jul  5 00:06 name_gender.csv
-rw-r--r--@ 1 giho.han  staff     519 Jul  5 00:06 main.py
-rw-r--r--@ 1 giho.han  staff  136855 Jul  5 00:06 customer-orders.csv
-rw-r--r--@ 1 giho.han  staff   62728 Jul  5 00:06 1800.csv
drwxr-xr-x@ 9 giho.han  staff     288 Jul  4 18:45 venv
-rw-r--r--@ 1 giho.han  staff  146855 Apr 10  2022 customer-orders.csv.1
-rw-r--r--@ 1 giho.han  staff  146855 Apr 10  2022 customer-orders.csv.2


In [7]:
!head -5 transfer_cost.txt

On 2021-01-04 the cost per ton from 85001 to 85002 is $28.32 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85004 is $25.68 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85007 is 19.86 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85007 is 20.52 at Haul Today
On 2021-01-04 the cost per ton from 85001 to 85010 is 20.72 at Haul Today


In [8]:
import pyspark.sql.functions as f
from pyspark.sql.types import *

schema = StructType([StructField("text",StringType(),True)])
trans_df = spark.read.schema(schema).text('transfer_cost.txt')


In [10]:
trans_df.show(truncate=False)

+---------------------------------------------------------------------------+
|text                                                                       |
+---------------------------------------------------------------------------+
|On 2021-01-04 the cost per ton from 85001 to 85002 is $28.32 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85004 is $25.68 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85007 is 19.86 at ABC Hauling |
|On 2021-01-04 the cost per ton from 85001 to 85007 is 20.52 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85010 is 20.72 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85012 is $18.98 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85013 is 26.64 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85020 is 26.34 at ABC Hauling |
|On 2021-01-04 the cost per ton from 85001 to 85021 is $20.15 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85002 to 85001 is 21.57 at 

In [24]:
from pyspark.sql.functions import regexp_extract

regex_str = r'On (\S+) the cost per ton from (\d+) to (\d+) is (\S+) at (.*)'
df_with_col = trans_df\
    .withColumn('week',regexp_extract('text',regex_str,1))\
    .withColumn('departure_zipcode',regexp_extract(col("text"),regex_str,2))\
    .withColumn('arrival_zipcode',regexp_extract(trans_df.text,regex_str,3))\
    .withColumn('cost',regexp_extract(col('text'),regex_str,4))\
    .withColumn('vendor',regexp_extract(col('text'),regex_str,5))

NameError: name 'col' is not defined

In [30]:
from pyspark.sql.functions import regexp_extract

regex_str = r'On (\S+) the cost per ton from (\d+) to (\d+) is (\S+) at (.*)'
df_with_col = trans_df\
    .withColumn('week',regexp_extract('text',regex_str,1))\
    .withColumn('departure_zipcode',regexp_extract('text',regex_str,2))\
    .withColumn('arrival_zipcode',regexp_extract(trans_df.text,regex_str,3))\
    .withColumn('cost',regexp_extract(trans_df.text,regex_str,4))\
    .withColumn('vendor',regexp_extract(trans_df.text,regex_str,5))

In [31]:
 df_with_col.printSchema()

root
 |-- text: string (nullable = true)
 |-- week: string (nullable = true)
 |-- departure_zipcode: string (nullable = true)
 |-- arrival_zipcode: string (nullable = true)
 |-- cost: string (nullable = true)
 |-- vendor: string (nullable = true)



In [32]:
df_with_col.show()

+--------------------+----------+-----------------+---------------+------+-----------+
|                text|      week|departure_zipcode|arrival_zipcode|  cost|     vendor|
+--------------------+----------+-----------------+---------------+------+-----------+
|On 2021-01-04 the...|2021-01-04|            85001|          85002|$28.32|ABC Hauling|
|On 2021-01-04 the...|2021-01-04|            85001|          85004|$25.68|ABC Hauling|
|On 2021-01-04 the...|2021-01-04|            85001|          85007| 19.86|ABC Hauling|
|On 2021-01-04 the...|2021-01-04|            85001|          85007| 20.52| Haul Today|
|On 2021-01-04 the...|2021-01-04|            85001|          85010| 20.72| Haul Today|
|On 2021-01-04 the...|2021-01-04|            85001|          85012|$18.98|ABC Hauling|
|On 2021-01-04 the...|2021-01-04|            85001|          85013| 26.64| Haul Today|
|On 2021-01-04 the...|2021-01-04|            85001|          85020| 26.34|ABC Hauling|
|On 2021-01-04 the...|2021-01-04|          

In [33]:
final_df = df_with_col.drop('text')


In [35]:
final_df.write.csv('extract2.csv')

In [38]:
!ls -tl extract2.csv/

total 312
-rw-r--r--@ 1 giho.han  staff       0 Jul  5 02:20 _SUCCESS
-rw-r--r--@ 1 giho.han  staff  156423 Jul  5 02:20 part-00000-35cfa437-c7f0-4f89-98cc-e22c9cc94e65-c000.csv


In [39]:
!head -5 extract2.csv/part-00000-35cfa437-c7f0-4f89-98cc-e22c9cc94e65-c000.csv


2021-01-04,85001,85002,$28.32,ABC Hauling
2021-01-04,85001,85004,$25.68,ABC Hauling
2021-01-04,85001,85007,19.86,ABC Hauling
2021-01-04,85001,85007,20.52,Haul Today
2021-01-04,85001,85010,20.72,Haul Today


In [40]:
final_df.write.format('json').save('extracted.json')

In [42]:
!ls -lt extracted.json/

total 856
-rw-r--r--@ 1 giho.han  staff       0 Jul  5 02:20 _SUCCESS
-rw-r--r--@ 1 giho.han  staff  436305 Jul  5 02:20 part-00000-61e83676-cb49-43ff-a73d-2ce0a82abbf1-c000.json
